<h1>(pseudo) Glicko Rating Algorithm</h1>
Ever wondered what that scene from "The Social Network" where Edwardo writes an algorithm on Mark's dorm room window, really meant? The algorithm, popularly a chess rating algorithm has been used over the years many times by networking and dating sites as popular as Tinder among many others. Let's study a version of the algorithm made by me as an interesting side project.
There are 3 main parameters that I've implemented-
<ol>
    <li> Erratic Behaviour(1 to 1000): because the real world has real ups and downd unexplainable by algorithms</li>
    <li> Decrease Ratio(1 to 10): how much do you want to penalize a player for his failure to match up?</li>
    <li> Number of Tests(However much patience you have. Keep it around 100): the number of players in the game. Keep this one low if you expect faster results</li>
</ol>
Insert the values to see a graph of all the players (competing with each other) and getting to the top over time

The model is based around social media success and rating over time. So don't be surprised when everyone reaches the top eventually! Monitor how each parameter changes the direction and momentum of the players.
    
Thank you for your viewing!

In [1]:
import math
import pandas as pd
import numpy as np
import random
import time
from scipy.interpolate import make_interp_spline
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
from matplotlib.widgets import Slider, Button
from IPython.display import clear_output



In [2]:
class Person:
    def __init__(self, name, exp,carr_id,jt_id,rating):
        self.name = name
        self.exp = exp
        self.carr_id=carr_id
        self.jt_id=jt_id
        self.rating=rating
    
    def disp(self):
        print("\nName is: ",self.name)
        print("Experience is: ",self.exp)
        print("Career ID is: ",self.carr_id)
        print("Current job title ID is: ",self.jt_id)
        print("Rating is:", self.rating)
        
        print("----------")
    def core_glicko(self, s,ropp,erratic_behavior,decrease_ratio):
        rnot=self.rating
        ini=rnot
        
        q=0.00575646273
        pi=3.141592653
        rd=350
        grd=1/(math.sqrt(1+1/(3*q*q*rd*rd/(pi*pi))))


        e=1/(1+pow(10,grd*(rnot-ropp)/(-400)))
        dsquared=1/(q*q*grd*grd*e*(1-e))
        r= rnot+q/((1/rd*rd)+1/dsquared)*grd*(s-e)
        
        
        ch=erratic_behavior*(r-ini)
        if(abs(ch)>3):
            ch=3
        if(ch<0):
            ch=ch/decrease_ratio
        else:
            if(self.rating>20):
                ch=ch/3
        if(ini+ch>=25):
            self.rating=25
        elif(ini+ch<=0):
            self.rating=0
        else:
            self.rating=ini+ch
    
        #if s==1:
        #    print(self.name, " was swiped right on, his/her score is:")
        #elif s==0.5:
        #    print(self.name, " draws, his/her score is: ")
        #else:
        #    print(self.name, " was swiped left on, his/her score is: ")
        #print(self.rating,"points")
        
        #print("gained/lost ",ch," points")
        #print("_________________________________________________________________________________________")
    def swipe_left(self,ropp,erratic_behavior,decrease_ratio):
        #print("\n LEFT SWIPED!!!!")
        self.core_glicko(0,ropp,erratic_behavior,decrease_ratio)
    def swipe_right(self,ropp,erratic_behavior,decrease_ratio):
        #print("\n RIGHT SWIPED!!!!")
        self.core_glicko(1,ropp,erratic_behavior,decrease_ratio)
    def match(self,ropp,erratic_behavior,decrease_ratio):
        self.core_glicko(0.5,ropp,erratic_behavior,decrease_ratio)
    


In [3]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [4]:
def eval(p,px,mean,erratic_behavior,decrease_ratio):
    if(p==px):
        arr.append(p.rating)
    x=3+mean/10
    y=3-mean/10
    if(px.rating-p.rating<x or px.rating-p.rating>(-y)):
        if(p.rating>=px.rating):
            choicee=random.random()
            if(choicee<0.75):
                p.swipe_right(px.rating,erratic_behavior,decrease_ratio)
            else:
                p.swipe_left(px.rating,erratic_behavior,decrease_ratio)
        else:
            choicee=random.random()
            if(choicee<0.75):
                p.swipe_left(px.rating,erratic_behavior,decrease_ratio)
            else:
                p.swipe_right(px.rating,erratic_behavior,decrease_ratio)

In [5]:


def testRun(kx,erratic_behavior,decrease_ratio):
    for ind,dt in df.iterrows():
        persona.append(Person(dt['Name'],dt['Years_in_Experience'],dt["company"], dt["Title"],dt["Rating"]))
    

    printProgressBar(0, 200, prefix = 'Progress:', suffix = 'Complete', length = 50)

    i=0
    mean=0
    for i in range(0,kx):
        for  px in persona[0:i]:

            arr=[]
            for p in persona[0:i]:


                eval(p,px,mean,erratic_behavior,decrease_ratio)

                arr.append(p.rating)

            arr_sum.append(arr)
            mean=np.mean(arr)


        printProgressBar(i + 1, kx, prefix = 'Progress:', suffix = 'Complete', length = 50)
        i=i+1
    #print(arr_sum)
    return len(arr_sum)
   
        

In [6]:
def process_everything(): 
    df1 = pd.DataFrame(arr_sum) 
    df1=df1.T
    df1.fillna(0, inplace=True)
    #display(df1)
    df_r = df1.to_numpy()
    arr_sup=df_r
    #print(len_of_x)
    return arr_sup
    

In [7]:




def plot_swipe(len_of_x,arr_sup,len_of_test):
    X = np.arange(len_of_x)

    c=['r','g','b','c','y']





    arrx=[]
    for i in range(0,len_of_test-1):
        arrx.append(random.choice(arr_sup))

    for a in arrx:
        X_Y_Spline = make_interp_spline(X, a)
        X_ = np.linspace(X.min(), X.max(), 10000)
        Y_ = X_Y_Spline(X_)
        ratio = .3

        plt.plot(X_, Y_,color=c[random.choice(range(0,5))],label=str("lmao"))



    plt.title("Rating over 50 days (1 day= 20 swipes)")
    plt.xlabel("days")
    plt.ylabel("Rating")
    plt.show()
    
    lol=0
    for a in arr_sup:
        if(lol>5):
            break
        if(a[0]<5):
            X_Y_Spline = make_interp_spline(X, arr_sup[random.choice(range(0,len_of_test-1))])
            X_ = np.linspace(X.min(), X.max(), 10000)
            Y_ = X_Y_Spline(X_)

            plt.plot(X_, Y_,color=c[random.choice(range(0,5))],label=str("lmao"))
            lol=lol+1


    plt.title("Rating over 50 days (1 day= 30 swipes)")
    plt.xlabel("days")
    plt.ylabel("Rating")
    plt.show()


In [8]:
def set_variables():
    
    global df
    global persona
    global arr_sum
    global arr_sup
    global arr
    
    df = pd.read_csv('rating.csv')
    persona=[]
    arr_sum=[]
    arr_sup=[]

    rows, cols = (0, 0)
    arr = [[0 for i in range(cols)] for j in range(rows)]

<h1>  </h1>

In [ ]:
try_again=True
while(try_again):
    clear_output(wait=False)
    eb, dc, lot=0,0,0
    eb=input("Insert Erratic Behaviour: ")
    dc=input("Insert Decrease Ratio: ")
    lot=input("Increase Length of Test: ")

    erratic_behavior=int(eb)             #1 to 1000
    decrease_ratio=int(dc)         #1 to 10
    len_of_test=int(lot)           #50 to 250

    set_variables()
    size_of_mat=testRun(len_of_test,erratic_behavior,decrease_ratio)
    data_of_swipe=process_everything()

    plot_swipe(size_of_mat,data_of_swipe,len_of_test)
    ta=input("Try again? (y/n)")
    try_again=True if ta=="y" else False